# Crear la clase  Target

## Ejercicio 1

Cree el archivo competencia_01.csv, usando el competencia_01_crudo.csv con una variable adicional llamada clase_ternaria, que contenga las categorías **CONTINUA, BAJA+1 y BAJA+2**

Ayudita
Para practicar el muy útil y necesario lenguaje **SQL**, vamos a utilizar una base de datos **OLAP** llamada **DuckDB**.

La documentación la puede encontrar [aquí](https://duckdb.org/docs/archive/0.8.1/sql/introduction) Procedemos a instalarla, esto se debe ejecutar una sola vez

In [1]:
%%bash
pip install duckdb
pip install jupysql
pip install duckdb-engine


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


Configuramos el entorno de ejecución. Si ya tiene todo instalado, solo necesita ejecutar esta celda para empezar a usar **duckdb**

In [2]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb://

Y ya podemos usar **SQL** dentro de una notebook!

In [3]:
%%sql
select 'hola mundo'

,'hola mundo'
0,hola mundo


Para cargar el archivo `.csv` a una tabla pasando su ubicación a través de la variable `dataset_path`:

In [4]:
dataset_path = '/home/bita/dmeyf/datasets/'
dataset_file = 'competencia_01_crudo.csv'

In [5]:
%%sql
create or replace table competencia_01_crudo as
select
    *
from read_csv_auto("{{dataset_path + dataset_file}}")

,Success


Hagamos unas queries básicas para comprobar que todo esta funcionando bien.

In [6]:
%sql select * from competencia_01_crudo limit 5

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,249221109,202101,1,0,0,61,300,1664.94,17994.40,1605.81,...,0.0,4,0.0,-37098.90,0.0,7867,118514.88,10,0,76538.25
1,249221468,202101,1,0,0,53,37,4879.78,18640.95,1149.08,...,0.0,4,0.0,-9864.93,0.0,1124,31427.14,14,0,0.00
2,249223005,202101,1,0,0,48,202,1828.35,37789.01,610.12,...,0.0,4,0.0,-32119.08,0.0,3486,855.11,2,0,19389.69
3,249228180,202101,1,0,0,66,320,175.87,2600.67,276.22,...,0.0,4,0.0,-10891.30,0.0,8925,7517.73,6,0,516.12
4,249232117,202101,1,0,0,79,373,16600.69,32270.07,16367.13,...,0.0,4,0.0,-43610.75,0.0,8607,37309.25,5,0,3084.99


In [7]:
%%sql
select
    foto_mes
    , count(*) as cantidad -- cuenta cuantos casos hay en cada foto_mes
                           -- y lo guarda en un campo llamado cantidad
from competencia_01_crudo
group by foto_mes

,foto_mes,cantidad
0,202101,162026
1,202102,162646
2,202103,163685
3,202104,164090
4,202105,164623
5,202106,164876


Perfecto, ahora cree una nueva tabla con la variable adicional que se le pide.

In [8]:
%%sql
CREATE OR REPLACE TABLE clientes_por_mes AS
SELECT DISTINCT numero_de_cliente, foto_mes
FROM competencia_01_crudo;

,Success


In [9]:
%%sql
-- Crear una tabla temporal para obtener el estado de los clientes
CREATE OR REPLACE TABLE clientes_estado AS
WITH clientes_foto_mes AS (
    SELECT
        c1.numero_de_cliente,
        c1.foto_mes AS foto_mes_actual,
        c2.foto_mes AS foto_mes_siguiente,
        c3.foto_mes AS foto_mes_dos_meses
    FROM clientes_por_mes c1
    LEFT JOIN clientes_por_mes c2
        ON c1.numero_de_cliente = c2.numero_de_cliente
        AND c2.foto_mes = c1.foto_mes + 1
    LEFT JOIN clientes_por_mes c3
        ON c1.numero_de_cliente = c3.numero_de_cliente
        AND c3.foto_mes = c1.foto_mes + 2
)
SELECT
    numero_de_cliente,
    foto_mes_actual,
    CASE
        WHEN foto_mes_siguiente IS NOT NULL AND foto_mes_dos_meses IS NOT NULL THEN 'CONTINUA'
        WHEN foto_mes_siguiente IS NULL AND foto_mes_dos_meses IS NOT NULL THEN 'BAJA+1'
        WHEN foto_mes_siguiente IS NULL AND foto_mes_dos_meses IS NULL THEN 'BAJA+2'
        ELSE NULL
    END AS clase_ternaria
FROM clientes_foto_mes;


,Success


In [10]:
%%sql
CREATE OR REPLACE TABLE competencia_01 AS
SELECT c.*, e.clase_ternaria
FROM competencia_01_crudo c
LEFT JOIN clientes_estado e
    ON c.numero_de_cliente = e.numero_de_cliente
    AND c.foto_mes = e.foto_mes_actual;

,Success


In [11]:
%sql select * from competencia_01 limit 5

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
0,301268134,202103,1,0,0,50,322,26322.14,118447.18,1319.62,...,7,0.00,-67236.36,0.0,5221,50168.68,49.0,5.0,69019.32,CONTINUA
1,301272583,202103,1,0,0,59,322,2015.63,19569.12,2010.69,...,7,0.00,-18660.33,0.0,7440,19730.69,13.0,0.0,1020.51,CONTINUA
2,301274914,202103,1,0,0,68,133,1664.39,-3524.50,2305.61,...,1,0.00,-10325.59,0.0,3810,457.47,1.0,0.0,3038.07,CONTINUA
3,301280892,202103,1,0,0,46,182,6391.98,50678.79,1904.15,...,7,13085.94,-12558.11,0.0,4704,1977.16,4.0,0.0,1313.76,CONTINUA
4,301282429,202103,1,0,0,51,184,1925.65,18761.07,1592.20,...,1,0.00,NaN,NaN,5589,NaN,NaN,NaN,0.00,CONTINUA


## Ejercicio 1.1
* ¿Cuál es la nominalidad de cada clase?


In [12]:
%%sql
PIVOT competencia_01
on clase_ternaria
USING count(numero_de_cliente)
GROUP BY foto_mes

,foto_mes,BAJA+1,BAJA+2,CONTINUA
0,202101,1,635,160605
1,202102,5,785,160839
2,202103,3,1020,161681
3,202104,5,982,161914
4,202105,0,1192,0
5,202106,0,164876,0


In [13]:
%%sql
SELECT clase_ternaria,
       COUNT(*) AS cantidad_clientes
FROM competencia_01
GROUP BY clase_ternaria;

,clase_ternaria,cantidad_clientes
0,None,167403
1,BAJA+2,169490
2,CONTINUA,645039
3,BAJA+1,14


* ¿Cuál es la proporción del target?

## Guardar tabla en .csv

Para guardar a un `.csv` simplemente debe ejecutar la siguiente sentencia

In [15]:
%sql COPY competencia_01 TO '{{dataset_path}}competencia_01.csv' (FORMAT CSV, HEADER)

,Success
